In [2]:
WKDIR=/workspace/hraijc/BB_M7xNui_Assembly/03_FLYE
READDIR=/workspace/hraijc/BB_M7xNui_Assembly/02_cleanReads
DUPLEXREADS=${READDIR}/M7xNui_duplex_clean.fastq
SIMPLEXREADS=${READDIR}/M7xNui_simplex_clean.fastq
ALLREADS=${READDIR}/M7xNui_all_clean.fastq
READS5k=${READDIR}/M7xNui_min5k_clean.fastq

In [3]:
mkdir -p $WKDIR/log

In [4]:
cd $WKDIR

In [3]:
module load conda
conda activate hraijc_flye


(/workspace/appscratch/miniconda/hraijc_flye) 

: 1

In [4]:
python /workspace/hraijc/git_clones/Flye/bin/flye -v

2.9-b1774
(/workspace/appscratch/miniconda/hraijc_flye) 

: 1

In [ ]:
##FLYE MANUAL:
#For the most recent ONT data basecalled with Guppy5+ SUP use the new --nano-hq mode. Expected error rate is <5%.
#For Q20 data, use a combination of --nano-hq and --read-error 0.03.

In [ ]:
#Assemble Duplex reads denovo
#Use Flag --nano-hq AND --read-error 0.03
#bsub -J Flye_duplex  -o $WKDIR/log/Flye_duplex.out -e $WKDIR/log/Flye_duplex.err -n 17 \
#"python /workspace/hraijc/git_clones/Flye/bin/flye --nano-hq ${DUPLEXREADS} --out-dir ${WKDIR}/duplex --read-error 0.03 --genome-size 600m --threads 16 --iterations 2"

In [ ]:
#Assemble All reads denovo
#Use Flag --nano-hq 
bsub -J Flye_2k -o $WKDIR/log/Flye_2k.out -e $WKDIR/log/Flye_2k.err -n 17 -R rusage[mem=100000] \
"python /workspace/hraijc/git_clones/Flye/bin/flye --nano-hq ${ALLREADS} --out-dir ${WKDIR}/M7xNui_min2k_Flye --genome-size 600m --threads 16 --iterations 2"

In [5]:
#Assemble Min5kb reads denovo
#Use Flag --nano-hq 
bsub -J Flye_5k -o $WKDIR/log/Flye_5k.out -e $WKDIR/log/Flye_5k.err -n 17 -R rusage[mem=100000] \
"python /workspace/hraijc/git_clones/Flye/bin/flye --nano-hq ${READS5k} --out-dir ${WKDIR}/M7xNui_min5k_Flye --genome-size 600m --threads 16 --iterations 2"

(/workspace/appscratch/miniconda/hraijc_flye) (/workspace/appscratch/miniconda/hraijc_flye) Job <816740> is submitted to default queue <lowpriority>.
(/workspace/appscratch/miniconda/hraijc_flye) 

: 1

In [5]:
#Assemble Min2kb reads denovo 
#Use Flag --nano-raw
# Doing 4 iterations # Prob too many two should be fine.
# removing genome-size option.

sbatch << EOF
#!/bin/bash
#SBATCH -J Flye-bb
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=15G
#SBATCH --time=33:00:00

module load conda
conda activate hraijc_flye

python /workspace/hraijc/git_clones/Flye/bin/flye --nano-raw ${ALLREADS} --out-dir ${WKDIR}/M7xNui_min2k_Flyeraw --threads 24 --iterations 4
EOF


Submitted batch job 330732


### QC Assemblies

In [10]:
# QC the three assemblies.

sbatch << EOF
#!/bin/bash
#SBATCH -J Flye-bb
#SBATCH -o ${WKDIR}/Assemb_stat3.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=1:00:00

module load BBMap/38.33

statswrapper.sh in=${WKDIR}/M7xNui_min2k_Flye/assembly.fasta,${WKDIR}/M7xNui_min5k_Flye/assembly.fasta,${WKDIR}/M7xNui_min2k_Flyeraw/40-polishing/polished_3.fasta format=3
EOF

Submitted batch job 333008


In [3]:
module load BBMap/38.33
bsub -J Flye_QC -o ${WKDIR}/log/Flye_QC.log -e ${WKDIR}/log/Flye_QC.err \
"statswrapper.sh in=${WKDIR}/M7xNui_min2k_Flye/assembly.fasta,${WKDIR}/M7xNui_min5k_Flye/assembly.fasta format=3"
module unload BBMap/38.33

Job <820518> is submitted to default queue <lowpriority>.


In [4]:
module load BUSCO/v5.2.2
bsub -J M7xNui_min2k_Busco -o ${WKDIR}/log/M7xNui_min2k_Busco.log -e ${WKDIR}/log/M7xNui_min2k_Busco.err -n 25 \
"busco -i ${WKDIR}/M7xNui_min2k_Flye/assembly.fasta -l eudicots -o M7xNui_min2k_Busco -m geno -c 24 --datasets_version odb10"
module unload BUSCO

Loading BUSCO/v5.2.2
  Loading requirement: singularity/3
Job <822012> is submitted to default queue <lowpriority>.


In [ ]:
######OLD#####







In [ ]:
#Assemble All reads with >=Q15 denovo
#Use Flag --nano-hq 
bsub -J Flye_all_q15 -o $WKDIR/log/Flye_all_q15.out -e $WKDIR/log/Flye_all_q15.err -n 17 \
"python /workspace/hraijc/git_clones/Flye/bin/flye --nano-hq ${ALLQ15READS} --out-dir ${WKDIR}/all_q15 --genome-size 600m --threads 16 --iterations 2"

In [ ]:
#Assemble All reads with >=Q20 denovo USING read error flag!
#Use Flag --nano-hq AND --read-error 0.03
bsub -J FLYE_all_q20_re3  -o $WKDIR/log/FLYE_all_q20_re3.out -e $WKDIR/log/FLYE_all_q20_re3.err -n 17 \
"python /workspace/hraijc/git_clones/Flye/bin/flye --nano-hq ${ALLQ20READS} --out-dir ${WKDIR}/all_q20_re3 --read-error 0.03 --genome-size 600m --threads 16 --iterations 2"